In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
from transformers import BertTokenizer
import torch
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [16]:
data_path = "D:/Data/neural-punctuator/szeged/"
file_path = data_path + "szeged.txt"

In [17]:
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.readlines()

In [18]:
len(text), text[1]

(82100,
 ' A szállásunk egy Balaton melletti kis üdülőfaluban, Zamárdiban volt, a Postának az üdülőházában.\n')

In [19]:
from collections import OrderedDict

text = list(OrderedDict.fromkeys(text))
len(text)

80875

In [20]:
from sklearn.utils import shuffle

text = shuffle(text, random_state=0)

text[1]

' Ez jutott az eszébe önkéntelenül.\n'

In [21]:
text = [t.strip() for t in text]
text[1]

'Ez jutott az eszébe önkéntelenül.'

In [22]:
len(' '.join([t.strip() for t in text]).split(' '))

1265372

In [23]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-multilingual-uncased')

Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master


In [24]:
def clean_text(text):
    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    text = text.replace('-', ',')
    text = text.replace(';', '.')
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')

    text = re.sub(r'--\s?--', '', text)
    text = re.sub(r'\s+', ' ', text)

    text = text.replace(' ,', ',')
    
    text = re.sub(r',\s?,', ',', text)
    
    return text.strip().lower()

In [25]:
# cleaned_text = [clean_text(t) for t in text]

# for t, tc in zip(' '.join(text).split(' ')[:100], ' '.join(cleaned_text).split(' ')):
#     print(f"{t:20}\t{tc}")

In [26]:
text = [clean_text(t) for t in text]
text[1]

'Ez jutott az eszébe önkéntelenül.'

In [27]:
tokenizer.encode(".?,")

[101, 119, 136, 117, 102]

In [28]:
len(' '.join(text).split(' '))

1248861

In [29]:
tokenized = tokenizer.tokenize(' '.join(text))

In [30]:
tokenized[:5]

['az', 'egyes', '_', 'lesz', '##allo']

In [31]:
len(tokenized)

2771074

In [32]:
text2 = tokenizer.convert_tokens_to_string(tokenized)

In [33]:
text2 = [t for t in text2.split(' ') if t not in [',', '.', '?']]
len(text2)

1332371

In [34]:
for t1, t2 in zip(' '.join(text).split(' ')[:1000], text2):
    print(f'{t1:20}{t2}')         

Az                  az
Egyes_Leszállópályánakegyes
például             _
nem                 leszallopalyanak
ez                  peldaul
volt                nem
a                   ez
neve                volt
akkoriban,          a
Angliának           neve
vagy                akkoriban
Nagy,Britanniának   anglianak
nevezték,           vagy
bár                 nagy
Londont,            britannianak
ebben               neveztek
meglehetősen        bar
bizonyos            londont
volt                ebben
Winston,            meglehetosen
mindig              bizonyos
Londonnak           volt
hívták.             winston
Ez                  mindig
jutott              londonnak
az                  hivtak
eszébe              ez
önkéntelenül.       jutott
Könyvtár            az
megosztása          eszebe
közben              onkentelenul
(                   konyvtar
a                   megosztasa
tulajdonságlap      kozben
Sharing,            (
Megosztás           a
fülén               tulajdonsa

haza,               es
hogy                novekedeshordozo
gyorsabban          szegmenserol
haladjanak,         a
vagy                neuer
akik                _
rendszergazdaként   marktrol
az                  jelentette
otthoni             be
gépről              penteken
távfelügyeleti      a
pillantást          tozsdet
szeretnének         mukodteto
vetni               deutsche
rendszerük          _
valamely            borse
elemére.            _
A                   ag
napokban            az
utaznak.            uj
Az                  szabalyozas
AOL                 2001
az                  oktober
elmúlt              1
év                  jen
karácsonyát         lep
megelőző            eletbe
időszakban          hiszen
2,5_milliárd        ma
dolláros            az
forgalmat           otthonukban
bonyolított.        szamitogepet
Hasonló             mukodtetok
mértékben           nagy
1,25                resze
százalékra          mar
csökkent            nem
a                   azoknak

In [59]:
id2target = {-1: 0,
              119: 1, # .
              136: 2, # ?
              117: 3,  # ,
              -2: -1, # will be masked
             }
target2id = {value: key for key, value in id2target.items()}
    
    
def create_target(encoded):
    targets = []
    text = []

    target = -2 # Always mask after [CLS] token

    text.append(encoded[0])
    idx = 1
    while idx < len(encoded):
        word = encoded[idx]
        if word in id2target.keys():
            if word in (119,117) and \
            tokenizer._convert_id_to_token(encoded[idx-1]).lstrip('▁').isnumeric() and \
            tokenizer._convert_id_to_token(encoded[idx+1]).startswith('##') and \
            tokenizer._convert_id_to_token(encoded[idx+1]).strip('#').isnumeric():
                targets.append(-2) # prev number
#                 print(-2)
#                 print(f"{tokenizer._convert_id_to_token(word):15}\t", end="")
                text.append(word)                
                target = -2 # '. or ,'
            else:
                target = word
        else:
            if tokenizer._convert_id_to_token(word).startswith('##'):
                target = -2
            targets.append(target)
#             print(target)
#             print(tokenizer._convert_id_to_token(word), '\t', end="")
            
            target = -1
            text.append(word)
            
        idx += 1

    targets.append(target)

    targets = [id2target[t] for t in targets]

    return text, targets

In [60]:
train_n = 70_000
valid_n = 8_000

train_text = ' '.join(text[:train_n])
valid_text = ' '.join(text[train_n:train_n+valid_n])
test_text = ' '.join(text[train_n+valid_n:])

len(train_text.split(' ')), len(valid_text.split(' ')), len(test_text.split(' '))

(1080644, 123975, 44242)

In [61]:
len(text) - train_n - valid_n

2875

In [62]:
train_tokens = tokenizer.encode(train_text)
valid_tokens = tokenizer.encode(valid_text)
test_tokens = tokenizer.encode(test_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (2398794 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (274493 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (97793 > 512). Running this sequence through the model will result in indexing errors


In [63]:
train_tokens, train_targets = create_target(train_tokens)
valid_tokens, valid_targets = create_target(valid_tokens)
test_tokens, test_targets = create_target(test_tokens)

In [64]:
# For backward campatibility
train_tokens, train_targets = [train_tokens], [train_targets]
valid_tokens, valid_targets = [valid_tokens], [valid_targets]
test_tokens, test_targets = [test_tokens], [test_targets]

In [65]:
with open(data_path + 'train_data.pkl', 'wb') as f:
    pickle.dump((train_tokens, train_targets), f)
with open(data_path + 'valid_data.pkl', 'wb') as f:
    pickle.dump((valid_tokens, valid_targets), f)
with open(data_path + 'test_data.pkl', 'wb') as f:
    pickle.dump((test_tokens, test_targets), f)

In [66]:
(np.array(train_targets) >= 0).sum()

1153158

In [67]:
(np.array(train_targets) >= 0).sum(), (np.array(valid_targets) >= 0).sum(), (np.array(test_targets) >= 0).sum()

(1153158, 132224, 46992)

In [68]:
sum(((np.array(train_targets) >= 0).sum(), (np.array(valid_targets) >= 0).sum(), (np.array(test_targets) >= 0).sum()))

1332374

In [69]:
from collections import Counter

for tr in (train_targets, valid_targets, test_targets):
    c = Counter((t for targets in tr for t in targets))
    print('\t'.join([str(c[i]) for i in (1,2,3,0,-1)]))

81135	1681	126929	943413	1027975
9207	178	14505	108334	117453
3369	70	5141	38412	41890


In [70]:
sum((np.array(vt) != -1).sum() for vt in train_targets), \
sum((np.array(vt) != -1).sum() for vt in valid_targets), \
sum((np.array(vt) != -1).sum() for vt in test_targets)

(1153158, 132224, 46992)

In [71]:
sum((np.array(vt) == 2).sum() for vt in test_targets)

70

In [72]:
sum(len(vt) for vt in train_targets), \
sum(len(vt) for vt in valid_targets), \
sum(len(vt) for vt in test_targets)

(2181133, 249677, 88882)

In [73]:
word_idx = 0
s = []
for te, ta in zip(test_tokens[0][:], test_targets[0]):
#     print(tokenizer._convert_id_to_token(te), ta)
    if ta != -1:
        s.append(te)
        decoded = tokenizer.decode(s)
        print(f"{word_idx:3}. {decoded:20}\t{ta}\t{[tokenizer._convert_id_to_token(ss) for ss in s]}")
        word_idx += 1
        s = []
    else:
        s.append(te)

  0. [CLS] hala          	0	['[CLS]', 'hala']
  1. az                  	0	['az']
  2. urnak               	3	['ur', '##nak']
  3. nagyon              	0	['nagyon']
  4. jo                  	0	['jo']
  5. kepessegeim         	0	['kepes', '##sege', '##im']
  6. vannak              	3	['vannak']
  7. es                  	0	['es']
  8. nagyon              	0	['nagyon']
  9. szivesen            	0	['sz', '##ives', '##en']
 10. tanitanek           	0	['tani', '##tane', '##k']
 11. egyetemeken         	3	['egyetem', '##eken']
 12. foiskolakon         	0	['fois', '##kola', '##kon']
 13. is                  	0	['is']
 14. teologusokat        	3	['teolog', '##usok', '##at']
 15. filozofusokat       	1	['filozof', '##usok', '##at']
 16. ha                  	0	['ha']
 17. az                  	0	['az']
 18. ember               	0	['ember']
 19. mindig              	0	['mindig']
 20. ugy                 	0	['ugy']
 21. gondolkozna         	3	['gon', '##dol', '##ko', '##zna']
 22. mint               

1040. a                   	0	['a']
1041. hagyomanyos         	0	['hagyomanyos']
1042. (                   	0	['(']
1043. "                   	0	['"']
1044. hallhato            	0	['hall', '##hato']
1045. "                   	0	['"']
1046. )                   	0	[')']
1047. eloadasok           	0	['elo', '##adas', '##ok']
1048. eloadomuveszeit     	0	['elo', '##ado', '##mu', '##vesz', '##eit']
1049. es                  	0	['es']
1050. az                  	0	['az']
1051. audiovizualis       	0	['audio', '##vi', '##zu', '##alis']
1052. muvek               	0	['mu', '##vek']
1053. eloadomuveszeit     	0	['elo', '##ado', '##mu', '##vesz', '##eit']
1054. megilleto           	0	['meg', '##ille', '##to']
1055. jogok               	0	['jogo', '##k']
1056. kozott              	1	['kozott']
1057. az                  	0	['az']
1058. alapito             	0	['ala', '##pito']
1059. (                   	0	['(']
1060. k                   	0	['k']
1061. )                   	0	[')']
1062. feladata       

1882. terjesztes          	0	['ter', '##jes', '##zte', '##s']
1883. magaban             	0	['maga', '##ban']
1884. foglalja            	0	['fog', '##lal', '##ja']
1885. kulonosen           	0	['kulonosen']
1886. a                   	0	['a']
1887. mupeldany           	0	['mu', '##pel', '##dany']
1888. tulajdonjoganak     	0	['tul', '##aj', '##don', '##jo', '##ganak']
1889. atruhazasat         	0	['at', '##ruh', '##aza', '##sat']
1890. es                  	0	['es']
1891. a                   	0	['a']
1892. mupeldany           	0	['mu', '##pel', '##dany']
1893. berbeadasat         	3	['ber', '##bea', '##dasa', '##t']
1894. valamint            	0	['valamint']
1895. a                   	0	['a']
1896. mupeldanynak        	0	['mu', '##pel', '##dany', '##nak']
1897. az                  	0	['az']
1898. orszagba            	0	['orszag', '##ba']
1899. forgalomba          	0	['for', '##galom', '##ba']
1900. hozatali            	0	['hoz', '##atal', '##i']
1901. cellal              	0	['cell', '##al'

2755. kiadott             	0	['kiado', '##tt']
2756. altalanos           	0	['altalanos']
2757. utasitasoktol       	0	['uta', '##sita', '##sok', '##tol']
2758. kezdve              	0	['kezdve']
2759. az                  	0	['az']
2760. ujrairo             	0	['ujra', '##iro']
2761. csoport             	0	['csoport']
2762. altal               	0	['altal']
2763. vegrehajtott        	0	['veg', '##re', '##ha', '##jtott']
2764. utolso              	0	['utolso']
2765. simitasokig         	1	['sim', '##itas', '##oki', '##g']
2766. a                   	0	['a']
2767. baj                 	0	['ba', '##j']
2768. megis               	0	['megis']
2769. nagy                	3	['nagy']
2770. egy                 	0	['egy']
2771. belso               	0	['belso']
2772. vizsgalat           	3	['viz', '##s', '##gala', '##t']
2773. amelyet             	0	['amelyet']
2774. az                  	0	['az']
2775. ozvegy              	3	['oz', '##veg', '##y']
2776. gergely             	0	['ger', '##gely']
2777. _

3633. es                  	0	['es']
3634. teljesiteni         	1	['teljes', '##iten', '##i']
3635. a                   	0	['a']
3636. hirkozlesi          	0	['hir', '##ko', '##zle', '##si']
3637. _                   	0	['_']
3638. dontobizottsag      	0	['dont', '##obi', '##zott', '##sag']
3639. elfogadta           	0	['elf', '##oga', '##dta']
3640. a                   	0	['a']
3641. jelentos            	0	['jelentos']
3642. piaci               	0	['pia', '##ci']
3643. erovel              	0	['ero', '##vel']
3644. rendelkezo          	0	['rende', '##lke', '##zo']
3645. szolgaltatok        	0	['szo', '##lga', '##lta', '##tok']
3646. azonositasanak      	0	['azonos', '##itasa', '##nak']
3647. alapelveit          	1	['ala', '##pel', '##vei', '##t']
3648. ha                  	0	['ha']
3649. egyszer             	0	['egy', '##szer']
3650. arra                	0	['arra']
3651. setal               	0	['seta', '##l']
3652. es                  	0	['es']
3653. nincs               	0	['nincs']
365

...

43336. millio              	0	['millio']
43337. euro                	0	['euro']
43338. osszegu             	0	['ossze', '##gu']
43339. kotvenyt            	0	['kot', '##veny', '##t']
43340. bocsatott           	0	['boc', '##sato', '##tt']
43341. ki                  	1	['ki']
43342. mukodesukhoz        	0	['mu', '##kod', '##esu', '##kh', '##oz']
43343. tobb                	0	['tobb']
43344. tamogatast          	3	['tamo', '##gatas', '##t']
43345. a                   	0	['a']
43346. befektetok          	0	['be', '##fekt', '##eto', '##k']
43347. szamara             	0	['szamara']
43348. pedig               	0	['pedig']
43349. vonzobb             	0	['von', '##zo', '##bb']
43350. kedvezmenyeket      	0	['ked', '##vez', '##menyek', '##et']
43351. igenyelnek          	0	['igen', '##yel', '##nek']
43352. a                   	0	['a']
43353. vallalkozoi         	0	['valla', '##lk', '##ozo', '##i']
43354. ovezetek            	3	['ove', '##zete', '##k']
43355. ajanlasaikkal       	0	['ajan', '##l

44251. celkonyvtar         	0	['cel', '##kon', '##yv', '##tar']
44252. titkositva          	0	['tit', '##kos', '##it', '##va']
44253. volt                	3	['volt']
44254. e                   	1	['e']
44255. az                  	0	['az']
44256. unokatestverem      	0	['uno', '##kate', '##stve', '##rem']
44257. es                  	0	['es']
44258. en                  	0	['en']
44259. sokat               	0	['sok', '##at']
44260. jatszottunk         	1	['jatszott', '##unk']
44261. osszetalalkoztunk   	0	['ossze', '##tala', '##lk', '##oz', '##tun', '##k']
44262. az                  	0	['az']
44263. osztalyfonokunkkel  	0	['os', '##zta', '##ly', '##fono', '##kun', '##kkel']
44264. es                  	0	['es']
44265. egy                 	0	['egy']
44266. masik               	0	['masik']
44267. csapattal           	1	['csapat', '##tal']
44268. voltak              	0	['voltak']
44269. percek              	3	['per', '##cek']
44270. amikor              	0	['amikor']
44271. a                  

45002. nem                 	0	['nem']
45003. volt                	0	['volt']
45004. erre                	0	['erre']
45005. kepes               	1	['kepes']
45006. a                   	0	['a']
45007. haromszaz           	0	['harom', '##sza', '##z']
45008. banyasz             	0	['ban', '##yas', '##z']
45009. munkahelyenek       	0	['mun', '##kah', '##ely', '##enek']
45010. megszunesevel       	0	['meg', '##sz', '##unes', '##evel']
45011. jaro                	0	['jaro']
45012. intezkedest         	0	['inte', '##zke', '##dest']
45013. azzal               	0	['azzal']
45014. indokoljak          	3	['indo', '##kol', '##jak']
45015. hogy                	0	['hogy']
45016. nincs               	0	['nincs']
45017. piaca               	0	['pia', '##ca']
45018. a                   	0	['a']
45019. lencsehegyi         	0	['len', '##cs', '##eh', '##eg', '##yi']
45020. szennek             	1	['sz', '##enne', '##k']
45021. na                  	0	['na']
45022. tobb                	0	['tobb']
45023. se  

45916. is                  	0	['is']
45917. belole              	1	['belo', '##le']
45918. a                   	0	['a']
45919. windows             	0	['windows']
45920. _                   	0	['_']
45921. 2000                	0	['2000']
45922. kapcsolodhatunk     	0	['kap', '##cs', '##olo', '##dha', '##tun', '##k']
45923. virtualis           	0	['virtual', '##is']
45924. maganhalozathoz     	0	['maga', '##nha', '##lo', '##zat', '##hoz']
45925. ugy                 	3	['ugy']
45926. hogy                	0	['hogy']
45927. magat               	0	['magat']
45928. az                  	0	['az']
45929. internetet          	0	['internet', '##et']
45930. is                  	0	['is']
45931. modemen             	0	['mode', '##men']
45932. keresztul           	0	['keresztul']
45933. erjuk               	0	['er', '##juk']
45934. el                  	1	['el']
45935. ha                  	0	['ha']
45936. osszeadjuk          	0	['ossze', '##ad', '##juk']
45937. a                   	0	['a']
45938. fobb 

46948. hatarozatainak      	0	['hata', '##roz', '##ata', '##inak']
46949. keretei             	0	['ker', '##ete', '##i']
46950. kozott              	0	['kozott']
46951. egyeni              	0	['egyeni']
46952. felelosseggel       	0	['fele', '##los', '##seg', '##gel']
46953. vezeti              	0	['veze', '##ti']
46954. az                  	0	['az']
46955. egyesulest          	1	['egyes', '##ules', '##t']
46956. szamos              	0	['szamos']
46957. szakerto            	0	['sz', '##aker', '##to']
46958. szerint             	0	['szerint']
46959. torokorszag         	0	['torok', '##orszag']
46960. olyan               	0	['olyan']
46961. tortenelmi          	0	['tortenelmi']
46962. forduloponthoz      	0	['ford', '##ulo', '##pont', '##hoz']
46963. erkezett            	3	['erkezett']
46964. mint                	0	['mint']
46965. a                   	0	['a']
46966. huszas              	0	['hus', '##zas']
46967. evekben             	3	['evekben']
46968. amikor              	0	['amikor']
